In [378]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.impute import KNNImputer

In [379]:
train = pd.read_csv('train.csv', encoding='utf-8')
test = pd.read_csv('test.csv', encoding='utf-8')

In [380]:
train['Target'] = (train.Stage=='Closed Won').astype('int')
test['Target'] = (test['Sales_Contract_No'] != 'None').astype('int')

In [381]:
train = train[(train.Stage=='Closed Won')|(train.Stage=='Closed Lost')]

In [382]:
def bc(train_bc, test_bc):
    new_bc = []
    for bc in test_bc:
        if bc in train_bc:
            new_bc.append(bc)
        else:
            new_bc.append('None')
    return new_bc

In [383]:
test['Billing_Country'] = bc(train.Billing_Country.to_list(), test.Billing_Country.to_list())

In [384]:
def acomodar_territorios(df):
    lista_reg = []
    lista_ter = []

    for comb in (df.Region + '-' + df.Territory):
        reg, ter = comb.split('-')
        lista_reg.append(reg)

        if ter == 'Japan':
            lista_ter.append('Japan')
        else:
            lista_ter.append(ter)
    
    df.Region = lista_reg
    df.Territory = lista_ter

In [385]:
acomodar_territorios(train)
acomodar_territorios(test)

In [386]:
def ter(train_ter, test_ter):
    new_ter = []
    for ter in test_ter:
        if bc in train_ter:
            new_ter.append(ter)
        else:
            new_ter.append('None')
    return new_ter

In [387]:
test['Territory'] = ter(train.Territory.to_list(), test.Territory.to_list())

In [388]:
def set_dates(df):
    
    df.Opportunity_Created_Date = pd.to_datetime(df.Opportunity_Created_Date, errors='coerce')
    df.Planned_Delivery_Start_Date = pd.to_datetime(df.Planned_Delivery_Start_Date, errors='coerce')
    df.Planned_Delivery_End_Date = pd.to_datetime(df.Planned_Delivery_End_Date, errors='coerce')
    df.Account_Created_Date = pd.to_datetime(df.Account_Created_Date, errors='coerce')
    df.Last_Modified_Date = pd.to_datetime(df.Last_Modified_Date, errors='coerce')
    df.Quote_Expiry_Date = pd.to_datetime(df.Quote_Expiry_Date, errors='coerce') 
    

In [389]:
def create_monetary_features(df):
    
    df['ASP_sum_per_Opportunity'] = df.groupby('Opportunity_ID')['ASP'].transform('sum')
    df['ASP_converted_sum_per_Opportunity'] = df.groupby('Opportunity_ID')['ASP_(converted)'].transform('sum')
    df['Convertibility'] = df['ASP_converted_sum_per_Opportunity'] / df['ASP_sum_per_Opportunity']

    df['Total_Amount_USD'] = df['Convertibility']*df['Total_Amount']
    df['Total_Taxable_Amount_USD'] = df['Convertibility']*df['Total_Taxable_Amount']
    
    df.drop(columns=['ASP_sum_per_Opportunity', 'ASP_converted_sum_per_Opportunity'])
    
    df['Prod_Price_Mean'] = df.groupby('Opportunity_ID')['Total_Amount_USD'].transform('mean')
    df['Prod_Price_Median'] = df.groupby('Opportunity_ID')['Total_Amount_USD'].transform('median')
    df['Prod_Price_Max'] = df.groupby('Opportunity_ID')['Total_Amount_USD'].transform('max')
    df['Prod_Price_Min'] = df.groupby('Opportunity_ID')['Total_Amount_USD'].transform('min')
    
    df['Total_TRF'] = df.groupby('Opportunity_ID')['TRF'].transform('sum')
    df['Price_Per_TRF'] = df.Total_TRF / df.Total_Taxable_Amount_USD
    

In [390]:
def create_date_features(df):
    
    df['Quote_Length'] = df.Quote_Expiry_Date - df.Opportunity_Created_Date
    df.Quote_Length = df.Quote_Length.astype('timedelta64[D]')
    
    df['Quote_Days_Left'] = df.Quote_Expiry_Date - df.Last_Modified_Date
    df.Quote_Days_Left = df.Quote_Days_Left.astype('timedelta64[D]')
    
    df['Quote_Days_Over_Delivery'] = df.Quote_Expiry_Date - df.Planned_Delivery_End_Date
    df.Quote_Days_Over_Delivery = df.Quote_Days_Over_Delivery.astype('timedelta64[D]')
    
    df['Year_Creation'] = df.Opportunity_Created_Date.dt.year
    df['Month_Creation'] = df.Opportunity_Created_Date.dt.month
    df['Day_Creation'] = df.Opportunity_Created_Date.dt.day
    
    #Year delivery ya existe
    df['Month_Delivery'] = df.Planned_Delivery_Start_Date.dt.month
    df['Day_Delivery'] = df.Planned_Delivery_Start_Date.dt.day
    
    df['Days_Passed'] = df.Last_Modified_Date - df.Opportunity_Created_Date
    df.Days_Passed = df.Days_Passed.astype('timedelta64[D]')
    
    df['Wait_Time_Days'] = df.Planned_Delivery_Start_Date - df.Opportunity_Created_Date
    df.Wait_Time_Days = df.Wait_Time_Days.astype('timedelta64[D]')
    
    df['Delivery_Window'] = df.Planned_Delivery_End_Date - df.Planned_Delivery_Start_Date
    df.Delivery_Window = df.Delivery_Window.astype('timedelta64[D]')
    
    df['Account_Antiquity'] = df.Opportunity_Created_Date - df.Account_Created_Date
    df.Account_Antiquity = df.Account_Antiquity.astype('timedelta64[D]')
    

In [391]:
def create_owners_features(df):
    
    df['Last_Modified_By_Owners'] = ((df.Last_Modified_By==df.Opportunity_Owner)|\
                                     (df.Last_Modified_By==df.Account_Owner))
    
    df['Managed_By_One_Person'] = (df.Opportunity_Owner==df.Account_Owner)
    

In [392]:
def territory_features(df):
    
    df['Territory_Aux'] = df.Territory.replace(['NW America', 'NE America', \
                                               'SE America', 'Central US', 'SW America'], \
                                              'United States')
    
    df['Bills_Same_Country'] = df.Territory_Aux == df.Billing_Country
    
    df.drop(columns='Territory_Aux', inplace=True)

In [393]:
def hasData(t):
    for x in t:
        if x != 'None':
            return 1
    return 0

def featurize_low_data_columns(df):
    
    df['Product_Category_B'] = df.groupby('Opportunity_ID')['Product_Category_B'].transform(hasData)
    df['Price'] = df.groupby('Opportunity_ID')['Price'].transform(hasData)
    df['Size'] = df.groupby('Opportunity_ID')['Size'].transform(hasData)
    df['Product_Type'] = df.groupby('Opportunity_ID')['Product_Type'].transform(hasData)
    df['Brand'] = df.groupby('Opportunity_ID')['Brand'].transform(hasData)
    df['Currency'] = df.groupby('Opportunity_ID')['Currency'].transform(hasData)
    df['Source '] = df.groupby('Opportunity_ID')['Source '].transform(hasData)
    
    df['Or'] = ((df.Product_Category_B)|(df.Price)|(df.Size)\
                |(df.Product_Type)|(df.Brand)|(df.Currency)\
                |(df['Source '])).astype('bool')

In [394]:
def set_numerical_features(df):
    
    set_dates(df)
    create_monetary_features(df)
    create_date_features(df)
    create_owners_features(df)
    territory_features(df)
    featurize_low_data_columns(df)
    
    df.drop_duplicates(subset='Opportunity_ID', inplace=True)
    
    df = df.replace([np.inf, -np.inf], np.nan)
    
    return df.drop(columns=['ID', 'Submitted_for_Approval',
       'Account_Created_Date', 'Account_Name', 'Opportunity_Name',
       'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner',
       'Opportunity_Created_Date', 'Last_Activity',
       'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By',
       'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP',
       'ASP_(converted)_Currency', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Actual_Delivery_Date', 'TRF',
       'Total_Amount', 'Total_Taxable_Amount_Currency', 'Total_Taxable_Amount',
       'Prod_Category_A', 'ASP_sum_per_Opportunity', 'Quote_Type',
       'ASP_converted_sum_per_Opportunity', 'Total_Amount_USD', 'Source ', 
       'Product_Category_B', 'Price', 'Size', 'Product_Type', 'Brand', 'Currency'])

In [395]:
train_set = set_numerical_features(train)
train_set.drop(columns='Stage', inplace=True)

In [396]:
test_set = set_numerical_features(test)

In [397]:
features = ['Region', 'Territory', 'Bureaucratic_Code', 'Billing_Country', 
            'Account_Type', 'Opportunity_Type', 'Delivery_Terms', 'Total_Amount_Currency']

encoder = ce.TargetEncoder(cols=features, smoothing = 0.85)
encoder.fit(train[features],train['Target'])

TargetEncoder(cols=['Region', 'Territory', 'Bureaucratic_Code',
                    'Billing_Country', 'Account_Type', 'Opportunity_Type',
                    'Delivery_Terms', 'Total_Amount_Currency'],
              smoothing=0.85)

In [398]:
def set_categorical_features(df):
    df = df.join(encoder.transform(df[features]).add_suffix('_target'))
    
    return df.drop(columns=['Region', 'Territory', 'Bureaucratic_Code', 'Billing_Country', 
            'Account_Type', 'Opportunity_Type', 'Delivery_Terms', 'Total_Amount_Currency'])
    
train_set = set_categorical_features(train_set)
test_set = set_categorical_features(test_set)

In [399]:
train_fts = train_set.columns

imputer_train = KNNImputer()
imputer_train.fit(train_set)
train_set=pd.DataFrame(imputer_train.transform(train_set))

train_set.columns = train_fts

In [400]:
test_fts = test_set.columns

imputer_test = KNNImputer()
imputer_test.fit(test_set)
test_set = pd.DataFrame(imputer_test.transform(test_set))

test_set.columns = test_fts

In [401]:
train_set

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Opportunity_ID,Delivery_Year,Target,Convertibility,Total_Taxable_Amount_USD,Prod_Price_Mean,...,Bills_Same_Country,Or,Region_target,Territory_target,Bureaucratic_Code_target,Billing_Country_target,Account_Type_target,Opportunity_Type_target,Delivery_Terms_target,Total_Amount_Currency_target
0,1.0,1.0,1.0,1.0,0.0,2016.0,0.0,1.131096,5.964044e+06,5.964044e+06,...,0.0,0.0,0.463392,0.535927,0.595676,0.477157,0.488024,0.520281,0.479801,0.471323
1,0.0,0.0,0.0,0.0,1.0,2016.0,1.0,1.131094,5.455268e+04,5.455268e+04,...,0.0,0.0,0.463392,0.535927,0.595676,0.477157,0.488024,0.520281,0.479801,0.471323
2,0.0,0.0,0.0,0.0,2.0,2016.0,1.0,1.000000,8.386560e+04,8.386560e+04,...,1.0,1.0,0.446166,0.632723,0.595676,0.451985,0.298962,0.520281,0.547011,0.463701
3,1.0,0.0,1.0,0.0,3.0,2018.0,0.0,1.000000,7.421882e+06,7.421882e+06,...,1.0,1.0,0.446166,0.632723,0.268750,0.451985,0.298962,0.158183,0.449874,0.463701
4,1.0,0.0,1.0,0.0,4.0,2018.0,0.0,1.000000,1.335719e+07,1.335719e+07,...,1.0,1.0,0.446166,0.632723,0.268750,0.451985,0.298962,0.158183,0.449874,0.463701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9786,1.0,1.0,0.0,0.0,12799.0,2016.0,1.0,1.131087,4.543578e+05,4.543578e+05,...,0.0,1.0,0.463392,0.388889,0.595676,0.513017,0.682270,0.520281,0.479801,0.471323
9787,1.0,0.0,0.0,0.0,12800.0,2016.0,0.0,1.000000,2.133250e+07,1.066625e+07,...,0.0,0.0,0.446166,0.347490,0.268750,0.513017,0.298962,0.158183,0.547011,0.463701
9788,1.0,1.0,1.0,1.0,12801.0,2016.0,1.0,1.131096,3.390065e+05,1.130022e+05,...,1.0,1.0,0.463392,0.805556,0.595676,0.788732,0.298962,0.520281,0.479801,0.471323
9789,1.0,1.0,1.0,1.0,12802.0,2016.0,0.0,1.000000,0.000000e+00,2.346797e+06,...,1.0,0.0,0.446166,0.347490,0.595676,0.451985,0.298962,0.520281,0.547011,0.463701


In [402]:
test_set

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Opportunity_ID,Delivery_Year,Target,Convertibility,Total_Taxable_Amount_USD,Prod_Price_Mean,...,Bills_Same_Country,Or,Region_target,Territory_target,Bureaucratic_Code_target,Billing_Country_target,Account_Type_target,Opportunity_Type_target,Delivery_Terms_target,Total_Amount_Currency_target
0,1.0,1.0,1.0,1.0,10689.0,2019.0,1.0,1.131098,4.155868e+05,138528.944054,...,0.0,1.0,0.463392,0.535927,0.595676,0.513017,0.682270,0.702041,0.479801,0.471323
1,1.0,1.0,1.0,1.0,10690.0,2019.0,1.0,1.131096,8.571262e+05,171425.235575,...,0.0,1.0,0.463392,0.535927,0.595676,0.513017,0.682270,0.702041,0.479801,0.471323
2,1.0,1.0,0.0,0.0,10691.0,2019.0,1.0,1.000000,2.103750e+04,21037.500000,...,0.0,1.0,0.446166,0.535927,0.595676,0.451985,0.488024,0.688221,0.547011,0.463701
3,1.0,1.0,1.0,0.0,10692.0,2019.0,1.0,1.000000,2.169106e+06,361517.750000,...,0.0,1.0,0.446166,0.535927,0.283333,0.451985,0.682270,0.520281,0.547011,0.463701
4,0.0,0.0,0.0,0.0,10693.0,2019.0,1.0,1.000000,5.752500e+03,5752.500000,...,0.0,1.0,0.446166,0.535927,0.595676,0.451985,0.682270,0.688221,0.547011,0.463701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,1.0,1.0,0.0,0.0,12364.0,2019.0,1.0,1.000000,1.477500e+05,73875.000000,...,0.0,1.0,0.463392,0.535927,0.595676,0.545454,0.682270,0.520281,0.527778,0.463701
1563,1.0,1.0,0.0,0.0,12365.0,2019.0,1.0,1.000000,4.505490e+04,45054.900000,...,0.0,0.0,0.446166,0.535927,0.595676,0.451985,0.488024,0.520281,0.547011,0.463701
1564,1.0,1.0,1.0,1.0,12366.0,2019.0,1.0,1.000000,1.001220e+05,100122.000000,...,0.0,0.0,0.446166,0.535927,0.595676,0.451985,0.488024,0.520281,0.547011,0.463701
1565,1.0,1.0,0.0,0.0,12367.0,2019.0,0.0,1.000000,1.432200e+05,143220.000000,...,0.0,0.0,0.446166,0.535927,0.595676,0.451985,0.488024,0.688221,0.547011,0.463701


In [403]:
train_set.to_csv('train_prepared.csv', index=False)
test_set.to_csv('test_prepared.csv', index=False)